# PRAW
https://praw.readthedocs.io/en/stable/

</br>
https://github.com/praw-dev/praw/

# Pros
- live metadata including comments and replies 
- 


# Cons
### 1.The Reddit API has a limit of about 1000 items per listing, so you won't be able to get more than that with PRAW. 
> <h3>To get more posts, one option is PushShift.io, a third-party service that has more data on Reddit items. PushShift has an API wrapper called PSAW.</h3>
> https://stackoverflow.com/questions/60340997/praw-python-doesnt-show-every-submission-from-a-subreddit


### 2. For some posts, the replies are extracted as the comments before we look into the replies layer

> 


# Updates

### Get the link of the image
- Done, we include the image in the post and the image link in the comments 

### Check the limit
- the API limit is 100. Maybe we need to use BeautifulSoup to scrap the webpage

### replace_more (100)
- set the parameter to None

In [1]:
#!pip install praw
#!pip install --upgrade https://github.com/praw-dev/praw/archive/master.zip
import praw
import json
import time
import config 
import re
from tqdm.auto import tqdm

In [2]:
reddit = praw.Reddit(
    client_id= config.client_id,
    client_secret= config.client_secret,
    password= config.password,
    username= config.username,
    user_agent= config.user_agent,
)
print(reddit.read_only)

False


In [3]:
#the following functions can be packaged in the config.py
def try_else_None(sub_id):
    submission = reddit.submission(sub_id)
    try:
        res = submission.media_metadata
        return res
    except:
        return None
    
def get_image_from_submission(sub_id):
    submission = reddit.submission(sub_id)
    if try_else_None(sub_id) is None:
        return None
    else:
        img_url = []
        for i in submission.media_metadata.keys():
            img_url.append(submission.media_metadata.get(i).get('p')[-1].get('u'))
        return img_url
        
def findURL(string):
  
    # findall() has been used 
    # with valid conditions for urls in string
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex,string)      
    return [x[0] for x in url]

In [4]:
def get_reddits(subreddit = "wallstreetbets",limit = None):
    #cannot find parameter for ALL 
    
    start_time = time.time()
    
    total_submissions = dict()
    total_submissions['subreddit'] = subreddit
    posts = []
    for submission in reddit.subreddit(subreddit).top(limit=limit): 
        #print(type(submission.__dict__))
        # print(submission.__dict__)
        
        #first, try to include some info about the submission (which refers to a post)
        submission_dict = {
            'id':submission.id,
            'author':submission.author,#[submission.author.id,submission.author.name],
            'url':submission.url,
            'num_comments':submission.num_comments,
            'permalink':submission.permalink,
            'score':submission.score,
            'upvote_ratio':submission.upvote_ratio,
            'media_metadata':try_else_None(submission.id),
            'selftext':submission.selftext,
            'image':get_image_from_submission(submission.id)
            
        }
        comments_list = []
        submission.comments.replace_more(limit=None)
        for comment in submission.comments.list():
#             if isinstance(comment, praw.models.MoreComments):
#                 continue
            comment_dict = {}
            comment_dict['id'] = comment.id
            comment_dict['body'] = comment.body
            comment_dict['img_url'] = findURL(comment.body)
            
            try:
                reply_list = []
                for reply in comment.replies:
                    reply_list.append({'id': reply.id,'body':reply.body})
                    print(type(i), i.body, sep=' ')
                
                #comment_dict['replies'] = comment.replies
                comment_dict['replies'] = reply_list
                    
                    
                
            except:
                comment_dict['replies'] = []
            comments_list.append(comment_dict)
            
        submission_dict['comments'] = comments_list
        #print(json.dumps(submission_dict, ensure_ascii = False,indent=4, sort_keys=True))

        posts.append(submission_dict)
        
        
        
        
    total_submissions['posts'] = posts
    
    end_time = time.time()
    if round(end_time - start_time) < 60:
        print('elasped time is',round(end_time - start_time,2),'s')
    else:
        print('elasped time is',round((end_time - start_time)/60,2),'s')
    return total_submissions
    #return json.dumps(total_submissions, ensure_ascii = False,indent=4, sort_keys=True)

In [ ]:
%%time
result = get_reddits(subreddit = "news",limit = 2)

In [ ]:
result

# Pushshift

https://www.reddit.com/r/pushshift/comments/bcxguf/new_to_pushshift_read_this_faq/

In [ ]:
from psaw import PushshiftAPI
api = PushshiftAPI(reddit)

In [ ]:
# The `search_comments` and `search_submissions` methods return generator objects
gen = api.search_submissions(limit=100)
results = list(gen)

In [8]:
for submission in reddit.subreddit('wallstreetbets').top(limit=None): 
    #print(type(submission.__dict__))
    print(submission.award)
    break

    #first, try to include some info about the submission (which refers to a post)
#     submission_dict = {
#         'id':submission.id,
#         'author':submission.author,#[submission.author.id,submission.author.name],
#         'url':submission.url,
#         'num_comments':submission.num_comments,
#         'permalink':submission.permalink,
#         'score':submission.score,
#         'upvote_ratio':submission.upvote_ratio,
#         'media_metadata':try_else_None(submission.id),
#         'selftext':submission.selftext,
#         'image':get_image_from_submission(submission.id)

#     }

<bound method GildableMixin.award of Submission(id='l8rf4k')>


In [11]:
reddit.comment('i5xm9rr').

'![gif](emote|free_emotes_pack|no_mouth)'